#**Step 1: Install All the Required Packages**

In [ ]:
!python -m pip install --upgrade pip -q -U
!pip install -q -U scipy
!pip install -q -U transformers
!pip install accelerate -q -U
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

hub_model_path = "bragour/Camel-13b-chat-4bit"
cache_dir='.'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    hub_model_path,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    cache_dir=cache_dir)

tokenizer = AutoTokenizer.from_pretrained(hub_model_path,use_fast=True,trust_remote_code=True)



In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
from transformers import pipeline
prompt = "what is the capitale of palestine ?"
pipe = pipeline(task = "text-generation",model=model, tokenizer=tokenizer)
result = pipe(f"<s>[INST]{prompt}[/INST]")
print(result[0]['generated_text'])